In [ ]:
!pip install pytorch-lightning
!pip install pytorch-forecasting
!pip install scikit-learn==1.3.2
import copy
from pathlib import Path
import warnings
import lightning.pytorch as pl
import pytorch_forecasting as pf
from pytorch_forecasting.data import NaNLabelEncoder
from lightning.pytorch.callbacks import EarlyStopping, ModelSummary
from lightning.pytorch.loggers import TensorBoardLogger
import numpy as np
import pandas as pd
import json
import pickle
import joblib
import torch
import gc
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names, but StandardScaler was fitted with feature names")

from pytorch_forecasting import TimeSeriesDataSet
from pytorch_forecasting.models import TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import QuantileLoss

#from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters


np.random.seed(42)
torch.manual_seed(42)


def percentage_error(actual, predicted):
    res = np.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs(percentage_error(np.asarray(y_true), np.asarray(y_pred))))

def create_Time_Dataset(train, test):

    max_encoder_length = 48
    max_prediction_length = 24

    training_cutoff = len(train) - max_prediction_length


    # Define the source training dataset
    training = TimeSeriesDataSet(
        train[lambda x: x.time_index <= training_cutoff],
        time_idx="time_index",
        target="Electrical_Consumption/sqm",
        group_ids=["group_id"],
        max_encoder_length=max_encoder_length,
        max_prediction_length=max_prediction_length,
        static_categoricals=[],
        static_reals=["lat", "lng", "yearbuilt"],
        time_varying_known_categoricals=["month"],
        time_varying_known_reals=["time_index", "airTemperature", "cloudCoverage", "dewTemperature", "windDirection", "windSpeed"],
        time_varying_unknown_categoricals=[],
        time_varying_unknown_reals=["Electrical_Consumption/sqm"],
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
        allow_missing_timesteps=False,
        categorical_encoders={"month": NaNLabelEncoder(add_nan=True)},
    )

    # Define the source validation and testing dataset
    validation = TimeSeriesDataSet.from_dataset(training, train, predict=True, stop_randomization=True)
    testing = TimeSeriesDataSet.from_dataset(training, test)


    return training, validation, testing




def TFT_model(dataset):

    if dataset == 'source':
        params_file = source_params
        dataset = training_source
    elif dataset == 'target':
        params_file = target_params
        dataset = training_target
    elif dataset == 'target_2':
        params_file = target_2_params
        dataset = training_target_2

    model = TemporalFusionTransformer.from_dataset(
        dataset,
        hidden_size = params_file['hidden_size'],
        hidden_continuous_size = params_file['hidden_continuous_size'],
        attention_head_size = params_file['attention_head_size'],
        dropout = params_file['dropout'],
        learning_rate = params_file['learning_rate'],
        lstm_layers = 2,
        loss = QuantileLoss(),
        optimizer = "Adam",
        log_interval = 10,
        log_val_interval = 10,
        reduce_on_plateau_patience = 4
    )

    return model


def fit_predict_TFT(dataset, mode, type_):

    metrics_unscaled = []

    if type_ == '6M':
        dir_save = 'Models/TFT/6M'
    elif type_ == '12M':
        dir_save = 'Models/TFT/12M'

    if dataset == 'source':
        params_file = source_params
        batch_size = params_file['batch_size']
        train_dataloader = training_source.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
        val_dataloader = validation_source.to_dataloader(train=False, batch_size=batch_size*10, num_workers=0)
        test_dataloader = testing_source.to_dataloader(train=False, batch_size=batch_size, num_workers=0)
        y_test = test_source['Electrical_Consumption/sqm']
        sqm = sqm_source
        scaler = scaler_source

        pl.seed_everything(42)
        early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
        logger = TensorBoardLogger("lightning_logs", log_graph=False)

        model = TFT_model(dataset)

        trainer = pl.Trainer(
            accelerator = "gpu" if torch.cuda.is_available() else "cpu",
            devices=1,
            max_epochs = 40,
            enable_model_summary=True,
            gradient_clip_val = 0.1,
            callbacks = [early_stop_callback],
            logger = logger,
            enable_checkpointing = False,
            log_every_n_steps=10,  # Log every step
            num_sanity_val_steps=0  # To avoid running validation before training starts
        )

        trainer.fit(model, train_dataloader, val_dataloader)

        trainer.save_checkpoint("Models/TFT/12M/BASE models/EC_BASE_source.ckpt")

        model = TemporalFusionTransformer.load_from_checkpoint("Models/TFT/12M/BASE models/EC_BASE_source.ckpt")

        predictions_scaled_gpu = model.predict(test_dataloader)

        predictions_scaled = (predictions_scaled_gpu.cpu()).numpy()

        predictions_scaled = predictions_scaled[:,:1].ravel()

        predictions_scaled = predictions_scaled[1:]

        del model, train_dataloader, val_dataloader, test_dataloader
        torch.cuda.empty_cache()
        gc.collect()

    elif dataset == 'target':
        params_file = target_params
        batch_size = params_file['batch_size']
        train_dataloader = training_target.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
        val_dataloader = validation_target.to_dataloader(train=False, batch_size=batch_size*10, num_workers=0)
        test_dataloader = testing_target.to_dataloader(train=False, batch_size=batch_size, num_workers=0)
        y_test = test_target['Electrical_Consumption/sqm']
        sqm = sqm_target
        scaler = scaler_target

        pl.seed_everything(42)
        logger = TensorBoardLogger("lightning_logs", log_graph=False)

        if mode == 'BASE':
            early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
        else :
            early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=6, verbose=False, mode="min")

        if mode == 'BASE':

            model = TFT_model(dataset)

            trainer = pl.Trainer(
                accelerator = "gpu" if torch.cuda.is_available() else "cpu",
                devices=1,
                max_epochs = 40,
                enable_model_summary=True,
                gradient_clip_val = 0.1,
                callbacks = [early_stop_callback],
                logger = logger,
                enable_checkpointing = False,
                log_every_n_steps=10,  # Log every step
                num_sanity_val_steps=0  # To avoid running validation before training starts
            )

            trainer.fit(model, train_dataloader, val_dataloader)

            trainer.save_checkpoint(dir_save + "/BASE models/EC_BASE_target.ckpt")

            predictions_scaled_gpu = model.predict(test_dataloader)

            predictions_scaled = (predictions_scaled_gpu.cpu()).numpy()

            predictions_scaled = predictions_scaled[:,:1].ravel()

            predictions_scaled = predictions_scaled[1:]

            del model, train_dataloader, val_dataloader, test_dataloader
            torch.cuda.empty_cache()
            gc.collect()

        elif mode == 'TL1':
            model = TemporalFusionTransformer.load_from_checkpoint("Models/TFT/12M/BASE models/EC_BASE_source.ckpt")

            trainer = pl.Trainer(
                accelerator = "gpu" if torch.cuda.is_available() else "cpu",
                devices=1,
                max_epochs = 40,
                gradient_clip_val=0.1,
                enable_model_summary=True,
                callbacks = [early_stop_callback],
                logger = logger,
                enable_checkpointing = False,
                log_every_n_steps=10,  # Log every step
                num_sanity_val_steps=0  # To avoid running validation before training starts
            )

            #finetuning the whole model
            trainer.fit(model, train_dataloader, val_dataloader)

            trainer.save_checkpoint(dir_save + "/TL models/EC_TL1_target.ckpt")

            predictions_scaled_gpu = model.predict(test_dataloader)

            predictions_scaled = (predictions_scaled_gpu.cpu()).numpy()

            predictions_scaled = predictions_scaled[:,:1].ravel()

            predictions_scaled = predictions_scaled[1:]

            del model, train_dataloader, val_dataloader, test_dataloader
            torch.cuda.empty_cache()
            gc.collect()

        elif mode == 'TL2':
            model = TemporalFusionTransformer.load_from_checkpoint("Models/TFT/12M/BASE models/EC_BASE_source.ckpt")

            #finetuning variable selection layers, static enrichment layers and the gated layers associated with them
            for name, param in model.named_parameters():
                if "static_variable_selection" in name or "encoder_variable_selection" in name or "decoder_variable_selection" in name or "static_enrichment" in name or "post_attn_gate_norm" in name or "post_lstm_gate_norm" in name:
                    param.requires_grad = True
                else:
                    param.requires_grad = False

            trainer = pl.Trainer(
                accelerator = "gpu" if torch.cuda.is_available() else "cpu",
                devices=1,
                max_epochs = 40,
                gradient_clip_val=0.1,
                enable_model_summary=True,
                callbacks = [early_stop_callback],
                logger = logger,
                enable_checkpointing = False,
                log_every_n_steps=10,  # Log every step
                num_sanity_val_steps=0  # To avoid running validation before training starts
            )


            trainer.fit(model, train_dataloader, val_dataloader)

            trainer.save_checkpoint(dir_save + "/TL models/EC_TL2_target.ckpt")

            predictions_scaled_gpu = model.predict(test_dataloader)

            predictions_scaled = (predictions_scaled_gpu.cpu()).numpy()

            predictions_scaled = predictions_scaled[:,:1].ravel()

            predictions_scaled = predictions_scaled[1:]

            del model, train_dataloader, val_dataloader, test_dataloader
            torch.cuda.empty_cache()
            gc.collect()

        elif mode == 'TL3':
            model = TemporalFusionTransformer.load_from_checkpoint("Models/TFT/12M/BASE models/EC_BASE_source.ckpt")

            #finetuning lstm layers
            for name, param in model.named_parameters():
                if "lstm_encoder" in name or "lstm_decoder" in name:
                    param.requires_grad = True
                else:
                    param.requires_grad = False

            trainer = pl.Trainer(
                accelerator = "gpu" if torch.cuda.is_available() else "cpu",
                devices=1,
                max_epochs = 40,
                gradient_clip_val=0.1,
                enable_model_summary=True,
                callbacks = [early_stop_callback],
                logger = logger,
                enable_checkpointing = False,
                log_every_n_steps=10,  # Log every step
                num_sanity_val_steps=0  # To avoid running validation before training starts
            )


            trainer.fit(model, train_dataloader, val_dataloader)

            trainer.save_checkpoint(dir_save + "/TL models/EC_TL3_target.ckpt")

            predictions_scaled_gpu = model.predict(test_dataloader)

            predictions_scaled = (predictions_scaled_gpu.cpu()).numpy()

            predictions_scaled = predictions_scaled[:,:1].ravel()

            predictions_scaled = predictions_scaled[1:]

            del model, train_dataloader, val_dataloader, test_dataloader
            torch.cuda.empty_cache()
            gc.collect()

        elif mode == 'TL4':
            model = TemporalFusionTransformer.load_from_checkpoint("Models/TFT/12M/BASE models/EC_BASE_source.ckpt")

            #finetuning attention layers, static context enrichment layers and the gated layers associated with them
            for name, param in model.named_parameters():
                if "multihead_attn" in name or "static_context_enrichment" in name or "post_attn_gate_norm" in name:
                    param.requires_grad = True
                else:
                    param.requires_grad = False

            trainer = pl.Trainer(
                accelerator = "gpu" if torch.cuda.is_available() else "cpu",
                devices=1,
                max_epochs = 40,
                gradient_clip_val=0.1,
                enable_model_summary=True,
                callbacks = [early_stop_callback],
                logger = logger,
                enable_checkpointing = False,
                log_every_n_steps=10,  # Log every step
                num_sanity_val_steps=0  # To avoid running validation before training starts
            )


            trainer.fit(model, train_dataloader, val_dataloader)

            trainer.save_checkpoint(dir_save + "/TL models/EC_TL4_target.ckpt")

            predictions_scaled_gpu = model.predict(test_dataloader)

            predictions_scaled = (predictions_scaled_gpu.cpu()).numpy()

            predictions_scaled = predictions_scaled[:,:1].ravel()

            predictions_scaled = predictions_scaled[1:]

            del model, train_dataloader, val_dataloader, test_dataloader
            torch.cuda.empty_cache()
            gc.collect()


    elif dataset == 'target_2':
        params_file = target_2_params
        batch_size = params_file['batch_size']
        train_dataloader = training_target_2.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
        val_dataloader = validation_target_2.to_dataloader(train=False, batch_size=batch_size*10, num_workers=0)
        test_dataloader = testing_target_2.to_dataloader(train=False, batch_size=batch_size, num_workers=0)
        y_test = test_target_2['Electrical_Consumption/sqm']
        sqm = sqm_target_2
        scaler = scaler_target_2

        pl.seed_everything(42)
        logger = TensorBoardLogger("lightning_logs", log_graph=False)

        if mode == 'BASE':
            early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
        else :
            early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=6, verbose=False, mode="min")

        if mode == 'BASE':

            model = TFT_model(dataset)

            trainer = pl.Trainer(
                accelerator = "gpu" if torch.cuda.is_available() else "cpu",
                devices=1,
                max_epochs = 40,
                gradient_clip_val=0.1,
                enable_model_summary=True,
                callbacks = [early_stop_callback],
                logger = logger,
                enable_checkpointing = False,
                log_every_n_steps=10,  # Log every step
                num_sanity_val_steps=0  # To avoid running validation before training starts
            )

            trainer.fit(model, train_dataloader, val_dataloader)

            trainer.save_checkpoint(dir_save + "/BASE models/EC_BASE_target_2.ckpt")

            predictions_scaled_gpu = model.predict(test_dataloader)

            predictions_scaled = (predictions_scaled_gpu.cpu()).numpy()

            predictions_scaled = predictions_scaled[:,:1].ravel()

            predictions_scaled = predictions_scaled[1:]

            del model, train_dataloader, val_dataloader, test_dataloader
            torch.cuda.empty_cache()
            gc.collect()

        elif mode == 'TL1':
            model = TemporalFusionTransformer.load_from_checkpoint("Models/TFT/12M/BASE models/EC_BASE_source.ckpt")

            trainer = pl.Trainer(
                accelerator = "gpu" if torch.cuda.is_available() else "cpu",
                devices=1,
                max_epochs = 40,
                gradient_clip_val=0.1,
                enable_model_summary=True,
                callbacks = [early_stop_callback],
                logger = logger,
                enable_checkpointing = False,
                log_every_n_steps=10,  # Log every step
                num_sanity_val_steps=0  # To avoid running validation before training starts
            )

            #finetuning the whole model
            trainer.fit(model, train_dataloader, val_dataloader)

            trainer.save_checkpoint(dir_save + "/TL models/EC_TL1_target_2.ckpt")

            predictions_scaled_gpu = model.predict(test_dataloader)

            predictions_scaled = (predictions_scaled_gpu.cpu()).numpy()

            predictions_scaled = predictions_scaled[:,:1].ravel()

            predictions_scaled = predictions_scaled[1:]

            del model, train_dataloader, val_dataloader, test_dataloader
            torch.cuda.empty_cache()
            gc.collect()

        elif mode == 'TL2':
            model = TemporalFusionTransformer.load_from_checkpoint("Models/TFT/12M/BASE models/EC_BASE_source.ckpt")

            #finetuning variable selection layers, static enrichment layers and the gated layers associated with them
            for name, param in model.named_parameters():
                if "static_variable_selection" in name or "encoder_variable_selection" in name or "decoder_variable_selection" in name or "static_enrichment" in name or "post_attn_gate_norm" in name or "post_lstm_gate_norm" in name:
                    param.requires_grad = True
                else:
                    param.requires_grad = False


            trainer = pl.Trainer(
                accelerator = "gpu" if torch.cuda.is_available() else "cpu",
                devices=1,
                max_epochs = 40,
                gradient_clip_val=0.1,
                enable_model_summary=True,
                callbacks = [early_stop_callback],
                logger = logger,
                enable_checkpointing = False,
                log_every_n_steps=10,  # Log every step
                num_sanity_val_steps=0  # To avoid running validation before training starts
            )


            trainer.fit(model, train_dataloader, val_dataloader)

            trainer.save_checkpoint(dir_save + "/TL models/EC_TL2_target_2.ckpt")

            predictions_scaled_gpu = model.predict(test_dataloader)

            predictions_scaled = (predictions_scaled_gpu.cpu()).numpy()

            predictions_scaled = predictions_scaled[:,:1].ravel()

            predictions_scaled = predictions_scaled[1:]

            del model, train_dataloader, val_dataloader, test_dataloader
            torch.cuda.empty_cache()
            gc.collect()

        elif mode == 'TL3':
            model = TemporalFusionTransformer.load_from_checkpoint("Models/TFT/12M/BASE models/EC_BASE_source.ckpt")

            #finetuning lstm layers
            for name, param in model.named_parameters():
                if "lstm_encoder" in name or "lstm_decoder" in name:
                    param.requires_grad = True
                else:
                    param.requires_grad = False

            trainer = pl.Trainer(
                accelerator = "gpu" if torch.cuda.is_available() else "cpu",
                devices=1,
                max_epochs = 40,
                gradient_clip_val=0.1,
                enable_model_summary=True,
                callbacks = [early_stop_callback],
                logger = logger,
                enable_checkpointing = False,
                log_every_n_steps=10,  # Log every step
                num_sanity_val_steps=0  # To avoid running validation before training starts
            )


            trainer.fit(model, train_dataloader, val_dataloader)

            trainer.save_checkpoint(dir_save + "/TL models/EC_TL3_target_2.ckpt")

            predictions_scaled_gpu = model.predict(test_dataloader)

            predictions_scaled = (predictions_scaled_gpu.cpu()).numpy()

            predictions_scaled = predictions_scaled[:,:1].ravel()

            predictions_scaled = predictions_scaled[1:]

            del model, train_dataloader, val_dataloader, test_dataloader
            torch.cuda.empty_cache()
            gc.collect()

        elif mode == 'TL4':
            model = TemporalFusionTransformer.load_from_checkpoint("Models/TFT/12M/BASE models/EC_BASE_source.ckpt")

            #finetuning attention layers, static context enrichment layers and the gated layers associated with them
            for name, param in model.named_parameters():
                if "multihead_attn" in name or "static_context_enrichment" in name or "post_attn_gate_norm" in name:
                    param.requires_grad = True
                else:
                    param.requires_grad = False

            trainer = pl.Trainer(
                accelerator = "gpu" if torch.cuda.is_available() else "cpu",
                devices=1,
                max_epochs = 40,
                gradient_clip_val=0.1,
                enable_model_summary=True,
                callbacks = [early_stop_callback],
                logger = logger,
                enable_checkpointing = False,
                log_every_n_steps=10,  # Log every step
                num_sanity_val_steps=0  # To avoid running validation before training starts
            )


            trainer.fit(model, train_dataloader, val_dataloader)

            trainer.save_checkpoint(dir_save + "/TL models/EC_TL4_target_2.ckpt")

            predictions_scaled_gpu = model.predict(test_dataloader)

            predictions_scaled = (predictions_scaled_gpu.cpu()).numpy()

            predictions_scaled = predictions_scaled[:,:1].ravel()

            predictions_scaled = predictions_scaled[1:]

            del model, train_dataloader, val_dataloader, test_dataloader
            torch.cuda.empty_cache()
            gc.collect()

    y_test = y_test[48:-24]

    predictions_unscaled = ((scaler.inverse_transform(predictions_scaled.reshape(-1, 1)))*(sqm/100)).reshape(-1)
    true_measurements_unscaled = ((scaler.inverse_transform(y_test.values.reshape(-1, 1)))*(sqm/100)).reshape(-1)

    metrics_unscaled = []

    periods = [24, 7*24, 30*24]
    
    for period in periods:

        rmse_unscaled = np.sqrt(mean_squared_error(true_measurements_unscaled[0:period], predictions_unscaled[0:period]))
        mae_unscaled = mean_absolute_error(true_measurements_unscaled[0:period], predictions_unscaled[0:period])
        mape_unscaled = mean_absolute_percentage_error(true_measurements_unscaled[0:period], predictions_unscaled[0:period])
        r2_score_unscaled = r2_score(true_measurements_unscaled[0:period], predictions_unscaled[0:period])*100

        metrics_unscaled.append(rmse_unscaled)
        metrics_unscaled.append(mae_unscaled)
        metrics_unscaled.append(mape_unscaled)
        metrics_unscaled.append(r2_score_unscaled)

        print("Results for {} samples".format(period))

        print('Test Unscaled RMSE: {}'.format(rmse_unscaled))
        print('Test Unscaled MAE: {}'.format(mae_unscaled))
        print('Test Unscaled R2 Score: ',r2_score_unscaled)
        print('Test Unscaled MAPE: ',mape_unscaled)



    return predictions_unscaled, metrics_unscaled


#------------- MAKE PREDICTIONS FOR 12M DATASETS ---------------#


train_source = pd.read_pickle('Data/TFT Preprocessed data/12M/train_source.pkl')
test_source = pd.read_pickle('Data/TFT Preprocessed data/12M/test_source.pkl')
test_source = test_source.reset_index(drop=True)

train_target = pd.read_pickle('Data/TFT Preprocessed data/12M/train_target.pkl')
test_target = pd.read_pickle('Data/TFT Preprocessed data/12M/test_target.pkl')
test_target = test_target.reset_index(drop=True)

train_target_2 = pd.read_pickle('Data/TFT Preprocessed data/12M/train_target_2.pkl')
test_target_2 = pd.read_pickle('Data/TFT Preprocessed data/12M/test_target_2.pkl')
test_target_2 = test_target_2.reset_index(drop=True)


with open('Models/TFT/12M/Tuned Hyperparameters/source_params.json', 'r') as f:
    source_params = json.load(f)

with open('Models/TFT/12M/Tuned Hyperparameters/target_params.json', 'r') as f1:
    target_params = json.load(f1)

with open('Models/TFT/12M/Tuned Hyperparameters/target_2_params.json', 'r') as f2:
    target_2_params = json.load(f2)

scaler_source = joblib.load('TFT Scalers/12M/source_scaler.pkl')
scaler_target = joblib.load('TFT Scalers/12M/target_scaler.pkl')
scaler_target_2 = joblib.load('TFT Scalers/12M/target_2_scaler.pkl')


with open('Domain sqms/sqm_source.json', 'r') as f:
    sqm_source = json.load(f)

with open('Domain sqms/sqm_target.json', 'r') as f:
    sqm_target = json.load(f)

with open('Domain sqms/sqm_target_2.json', 'r') as f:
    sqm_target_2 = json.load(f)


training_source, validation_source, testing_source = create_Time_Dataset(train_source, test_source)
training_target, validation_target, testing_target = create_Time_Dataset(train_target, test_target)
training_target_2, validation_target_2, testing_target_2 = create_Time_Dataset(train_target_2, test_target_2)



predictions_base_source, metrics_base_source = fit_predict_TFT('source', 'BASE', '12M')

np.save('Models/TFT/12M/Predictions/predictions_base_source.npy', predictions_base_source)
with open('Models/TFT/12M/Metrics/metrics_base_source.pkl', 'wb') as file:
    pickle.dump(metrics_base_source, file)



predictions_base_target, metrics_base_target = fit_predict_TFT('target', 'BASE', '12M')

np.save('Models/TFT/12M/Predictions/predictions_base_target.npy', predictions_base_target)
with open('Models/TFT/12M/Metrics/metrics_base_target.pkl', 'wb') as file:
    pickle.dump(metrics_base_target, file)


predictions_TL1_target, metrics_TL1_target = fit_predict_TFT('target', 'TL1', '12M')

np.save('Models/TFT/12M/Predictions/predictions_TL1_target.npy', predictions_TL1_target)
with open('Models/TFT/12M/Metrics/metrics_TL1_target.pkl', 'wb') as file:
    pickle.dump(metrics_TL1_target, file)


predictions_TL2_target, metrics_TL2_target = fit_predict_TFT('target', 'TL2', '12M')

np.save('Models/TFT/12M/Predictions/predictions_TL2_target.npy', predictions_TL2_target)
with open('Models/TFT/12M/Metrics/metrics_TL2_target.pkl', 'wb') as file:
    pickle.dump(metrics_TL2_target, file)


predictions_TL3_target, metrics_TL3_target = fit_predict_TFT('target', 'TL3', '12M')

np.save('Models/TFT/12M/Predictions/predictions_TL3_target.npy', predictions_TL3_target)
with open('Models/TFT/12M/Metrics/metrics_TL3_target.pkl', 'wb') as file:
    pickle.dump(metrics_TL3_target, file)


predictions_TL4_target, metrics_TL4_target = fit_predict_TFT('target', 'TL4', '12M')

np.save('Models/TFT/12M/Predictions/predictions_TL4_target.npy', predictions_TL4_target)
with open('Models/TFT/12M/Metrics/metrics_TL4_target.pkl', 'wb') as file:
    pickle.dump(metrics_TL4_target, file)


predictions_base_target_2, metrics_base_target_2 = fit_predict_TFT('target_2', 'BASE', '12M')

np.save('Models/TFT/12M/Predictions/predictions_base_target_2.npy', predictions_base_target_2)
with open('Models/TFT/12M/Metrics/metrics_base_target_2.pkl', 'wb') as file:
    pickle.dump(metrics_base_target_2, file)


predictions_TL1_target_2, metrics_TL1_target_2 = fit_predict_TFT('target_2', 'TL1', '12M')

np.save('Models/TFT/12M/Predictions/predictions_TL1_target_2.npy', predictions_TL1_target_2)
with open('Models/TFT/12M/Metrics/metrics_TL1_target_2.pkl', 'wb') as file:
    pickle.dump(metrics_TL1_target_2, file)


predictions_TL2_target_2, metrics_TL2_target_2 = fit_predict_TFT('target_2', 'TL2', '12M')

np.save('Models/TFT/12M/Predictions/predictions_TL2_target_2.npy', predictions_TL2_target_2)
with open('Models/TFT/12M/Metrics/metrics_TL2_target_2.pkl', 'wb') as file:
    pickle.dump(metrics_TL2_target_2, file)


predictions_TL3_target_2, metrics_TL3_target_2 = fit_predict_TFT('target_2', 'TL3', '12M')

np.save('Models/TFT/12M/Predictions/predictions_TL3_target_2.npy', predictions_TL3_target_2)
with open('Models/TFT/12M/Metrics/metrics_TL3_target_2.pkl', 'wb') as file:
    pickle.dump(metrics_TL3_target_2, file)


predictions_TL4_target_2, metrics_TL4_target_2 = fit_predict_TFT('target_2', 'TL4', '12M')

np.save('Models/TFT/12M/Predictions/predictions_TL4_target_2.npy', predictions_TL4_target_2)
with open('Models/TFT/12M/Metrics/metrics_TL4_target_2.pkl', 'wb') as file:
    pickle.dump(metrics_TL4_target_2, file)


#------------- MAKE PREDICTIONS FOR 6M DATASETS ---------------#



train_target = pd.read_pickle('Data/TFT Preprocessed data/6M/train_target.pkl')
test_target = pd.read_pickle('Data/TFT Preprocessed data/6M/test_target.pkl')
test_target = test_target.reset_index(drop=True)


with open('Models/TFT/6M/Tuned Hyperparameters/target_params.json', 'r') as f1:
    target_params = json.load(f1)


scaler_target = joblib.load('TFT Scalers/6M/target_scaler.pkl')


with open('Domain sqms/sqm_target.json', 'r') as f:
    sqm_target = json.load(f)


training_target, validation_target, testing_target = create_Time_Dataset(train_target, test_target)


predictions_base_target, metrics_base_target = fit_predict_TFT('target', 'BASE', '6M')

np.save('Models/TFT/6M/Predictions/predictions_base_target.npy', predictions_base_target)
with open('Models/TFT/6M/Metrics/metrics_base_target.pkl', 'wb') as file:
    pickle.dump(metrics_base_target, file)


predictions_TL1_target, metrics_TL1_target = fit_predict_TFT('target', 'TL1', '6M')

np.save('Models/TFT/6M/Predictions/predictions_TL1_target.npy', predictions_TL1_target)
with open('Models/TFT/6M/Metrics/metrics_TL1_target.pkl', 'wb') as file:
    pickle.dump(metrics_TL1_target, file)


predictions_TL2_target, metrics_TL2_target = fit_predict_TFT('target', 'TL2', '6M')

np.save('Models/TFT/6M/Predictions/predictions_TL2_target.npy', predictions_TL2_target)
with open('Models/TFT/6M/Metrics/metrics_TL2_target.pkl', 'wb') as file:
    pickle.dump(metrics_TL2_target, file)


predictions_TL3_target, metrics_TL3_target = fit_predict_TFT('target', 'TL3', '6M')

np.save('Models/TFT/6M/Predictions/predictions_TL3_target.npy', predictions_TL3_target)
with open('Models/TFT/6M/Metrics/metrics_TL3_target.pkl', 'wb') as file:
    pickle.dump(metrics_TL3_target, file)


predictions_TL4_target, metrics_TL4_target = fit_predict_TFT('target', 'TL4', '6M')

np.save('Models/TFT/6M/Predictions/predictions_TL4_target.npy', predictions_TL4_target)
with open('Models/TFT/6M/Metrics/metrics_TL4_target.pkl', 'wb') as file:
    pickle.dump(metrics_TL4_target, file)

